# Laboratory - Atheism

## Inference for categorical data

In August of 2012, news outlets ranging from the [Washington Post](http://www.washingtonpost.com/national/on-faith/poll-shows-atheism-on-the-rise-in-the-us/2012/08/13/90020fd6-e57d-11e1-9739-eef99c5fb285_story.html) to the [Huffington Post](http://www.huffingtonpost.com/2012/08/14/atheism-rise-religiosity-decline-in-america_n_1777031.html) ran a story about the rise of atheism in America. The source for the story was a poll that asked people, “Irrespective of whether you attend a place of worship or not, would you say you are a religious person, not a religious person or a convinced atheist?” This type of question, which asks people to classify themselves in one way or another, is common in polling and generates categorical data. In this lab we take a look at the atheism survey and explore what’s at play when making inference about population proportions using categorical data.

## The survey

To access the press release for the poll, conducted by WIN-Gallup International, click on the following [link](https://github.com/OpenIntroStat/oilabs/blob/master/data-raw/atheism/Global_INDEX_of_Religiosity_and_Atheism_PR__6.pdf).

Take a moment to review the report then address the following questions.

### Exercise 1 - In the first paragraph, several key findings are reported. Do these percentages appear to be sample statistics (derived from the data sample) or population parameters?

It seems to me that these data come from a sample.

### Exercise 2 - The title of the report is “Global Index of Religiosity and Atheism”. To generalize the report’s findings to the global human population, what must we assume about the sampling method? Does that seem like a reasonable assumption?

We must assume random sampling was carried out when selecting the 50,000 partecipants, and it seems reasonable to me if proper stratification was also applied.

## The data

Turn your attention to Table 6 (pages 15 and 16), which reports the sample size and response percentages for all 57 countries. While this is a useful format to summarize the data, we will base our analysis on the original data set of individual responses to the survey. Load this data set into R with the following command.

In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from scipy import stats

from pathlib import Path

sns.set_theme()

if not Path("./atheism.csv").exists():
    ! curl https://www.openintro.org/book/statdata/atheism.csv >> atheism.csv

atheism_df = pd.read_csv(Path("atheism.csv"))
atheism_df.sample(10)

,nationality,response,year
47844,Tunisia,non-atheist,2012
148694,Canada,non-atheist,2005
117380,"Korea, Rep (South)",non-atheist,2012
4676,Azerbaijan,non-atheist,2012
146826,Austria,non-atheist,2005
71085,Ukraine,non-atheist,2005
2100,Armenia,non-atheist,2012
62844,Italy,non-atheist,2005
58339,Austria,non-atheist,2005
83828,Macedonia,non-atheist,2005


### Exercise 3 - What does each row of Table 6 correspond to? What does each row of atheism correspond to?

In Table 6 of the report we see how responses are distributed for each country (each row is a country with its statistics regarding religion in addition to the sample size used to gather those statistics). Each row in atheism is the respondent taking part in the survey.

To investigate the link between these two ways of organizing this data, take a look at the estimated proportion of atheists in the United States. Towards the bottom of Table 6, we see that this is 5%. We should be able to come to the same number using the `atheism` data.